In [1]:
import cv2
import numpy as np
from Person import Person
from utils import draw

In [1]:
%cd re3-tensorflow/
import cv2
import numpy as np
from tracker import re3_tracker

[WinError 2] The system cannot find the file specified: 're3-tensorflow/'
C:\Users\velmisov\Ilya\Kyrsovaya


ModuleNotFoundError: No module named 'tracker'

In [2]:
cd ..

C:\Users\Ilya\Kyrsach\Kyrsovaya


In [3]:
% cd ./darkflow-master/
from darkflow.net.build import TFNet
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
%matplotlib inline 

C:\Users\Ilya\Kyrsach\Kyrsovaya\darkflow-master


In [4]:
optinos = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.3,
    'gpu': 1.0
}

In [5]:
tfnet = TFNet(optinos)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.05086064338684082s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

In [9]:
def counting_people(name_of_video):
    global tracker
    cap = cv2.VideoCapture(name_of_video)
    w = cap.get(3)
    h = cap.get(4)
    frameArea = h*w
    areaTH = frameArea/250
    person = {}
    index_of_people = '1'
    fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
    
    kernelOp = np.ones((3,3), np.uint8)
    kernelCl = np.ones((11,11), np.uint8)
    
    ok, frame = cap.read()
    lines = draw.draw("Test_draw", frame)
    
    while(cap.isOpened()):
        ok, frame = cap.read()
        fgmask = fgbg.apply(frame)
        try:
            ret, imBin = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
            mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernelCl)
        except:
            print('EOF')
            break

        _, contours0, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for cnt in contours0:
            area = cv2.contourArea(cnt)
            if area > areaTH:
                x,y,w,h = cv2.boundingRect(cnt)
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                box = [x, y, x+w, y+h]
                M = cv2.moments(cnt)
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
                cv2.imshow("Test", frame[y:y+h,x:x+h])
                ###########
                new = True
                for key in person:
                    #print(key, person[key], type(person[key]))
                    if abs(x-person[key].get_x()) <= w and abs(y - person[key].get_y()) <= h:
                        new = False
                        person[key].update_coordinates(x, y, w, h)
                        #p = tracker.multi_track(list(person.keys()), frame[:,:,-1])
                        break
                if new == True:
                    p = Person.Person(index_of_people, x, y, h, w)
                    person[index_of_people] = p
                    #p = tracker.multi_track(list(person.keys()), frame[:,:,-1], {index_of_people : box})
                    index_of_people = str(int(index_of_people) + 1)
                ###########    
            
        try:
            cv2.rectangle(frame, lines['intresting_zone'][0], lines['intresting_zone'][2], (255,255,255), 1)
            cv2.line(frame, lines['red_up_line'][0], lines['red_up_line'][1], (0,0,255), 2)
            cv2.line(frame, lines['blue_down_line'][0], lines['blue_down_line'][1], (255,0,0), 2)
            cv2.imshow('Count people',frame)
        except:
            print('EOF')
            break
        k = cv2.waitKey(20) & 0xFF
        if k == 27:
            break
    cap.release()
    cv2.destroyAllWindows()
    return person, lines

In [6]:
cd ..

C:\Users\Ilya\Kyrsach\Kyrsovaya


In [ ]:
p, lines = counting_people('./data/peopleCounter.avi')

In [3]:
import Person
import time


cnt_up   = 0
cnt_down = 0

cap = cv2.VideoCapture('./data/peopleCounter.avi')

w = cap.get(3)
h = cap.get(4)
frameArea = h*w
areaTH = frameArea/250
print('Area Threshold', areaTH)

#Lineas de entrada/salida
line_up = int(2*(h/5))
line_down   = int(3*(h/5))

up_limit =   int(1*(h/5))
down_limit = int(4*(h/5))

print("Red line y:",str(line_down))
print("Blue line y:", str(line_up))
line_down_color = (255,0,0)
line_up_color = (0,0,255)
pt1 =  [0, line_down];
pt2 =  [w, line_down];
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
print(pts_L1)
pt3 =  [0, line_up];
pt4 =  [w, line_up];
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit];
pt6 =  [w, up_limit];
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit];
pt8 =  [w, down_limit];
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))

#Substractor de fondo
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True)

#Elementos estructurantes para filtros morfoogicos
kernelOp = np.ones((3,3),np.uint8)
kernelOp2 = np.ones((5,5),np.uint8)
kernelCl = np.ones((11,11),np.uint8)

#Variables
font = cv2.FONT_HERSHEY_SIMPLEX
persons = []
max_p_age = 5
pid = 1

while(cap.isOpened()):
    ret, frame = cap.read()
    for i in persons:
        i.age_one() #age every person one frame
    #########################
    #   PRE-PROCESAMIENTO   #
    #########################
    
    #Aplica substraccion de fondo
    fgmask = fgbg.apply(frame)
    fgmask2 = fgbg.apply(frame)

    #Binariazcion para eliminar sombras (color gris)
    try:
        ret,imBin= cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
        ret,imBin2 = cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)
        #Opening (erode->dilate) para quitar ruido.
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask2 = cv2.morphologyEx(imBin2, cv2.MORPH_OPEN, kernelOp)
        #Closing (dilate -> erode) para juntar regiones blancas.
        mask =  cv2.morphologyEx(mask , cv2.MORPH_CLOSE, kernelCl)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        print('UP:',cnt_up)
        print('DOWN:',cnt_down)
        break
    #################
    #   CONTORNOS   #
    #################
    
    # RETR_EXTERNAL returns only extreme outer flags. All child contours are left behind.
    _, contours0, hierarchy = cv2.findContours(mask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours0:
        area = cv2.contourArea(cnt)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################
            
            #Falta agregar condiciones para multipersonas, salidas y entradas de pantalla.
            
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)
            #results = tfnet.return_predict(frame[y:y+h,x:x+h,:])
            #print(results)
            new = True
            if cy in range(up_limit,down_limit):
                for i in persons:
                    if (abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h):
                        # el objeto esta cerca de uno que ya se detecto antes
                        new = False
                        i.updateCoords(cx,cy)   #actualiza coordenadas en el objeto and resets age
                        if i.going_UP(line_down,line_up) == True:
                            cnt_up += 1;
                            print("ID:",i.getId(),'crossed going up at',time.strftime("%c"))
                        elif i.going_DOWN(line_down,line_up) == True:
                            cnt_down += 1;
                            print("ID:",i.getId(),'crossed going down at',time.strftime("%c"))
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                        elif i.getDir() == 'up' and i.getY() < up_limit:
                            i.setDone()
                    if i.timedOut():
                        #sacar i de la lista persons
                        index = persons.index(i)
                        persons.pop(index)
                        del i     #liberar la memoria de i
                if new == True:
                    p = Person.MyPerson(pid,cx,cy, max_p_age)
                    persons.append(p)
                    pid += 1     
            #################
            #   DIBUJOS     #
            #################
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            
            #cv2.drawContours(frame, cnt, -1, (0,255,0), 3)
            
    #END for cnt in contours0
            
    #########################
    # DIBUJAR TRAYECTORIAS  #
    #########################
    for i in persons:
        if len(i.getTracks()) >= 2:
            pts = np.array(i.getTracks(), np.int32)
            pts = pts.reshape((-1,1,2))
            frame = cv2.polylines(frame,[pts],False,i.getRGB())
       
        cv2.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,1,i.getRGB(),1,cv2.LINE_AA)
        
    #################
    #   IMAGANES    #
    #################
    str_up = 'UP: '+ str(cnt_up)
    str_down = 'DOWN: '+ str(cnt_down)
    frame = cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
    frame = cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,0,0),1,cv2.LINE_AA)

    cv2.imshow('Frame',frame)
    #cv2.imshow('Mask',mask)    
    
    #preisonar ESC para salir
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
#END while(cap.isOpened())
    
#################
#   LIMPIEZA    #
#################
cap.release()
cv2.destroyAllWindows()

Area Threshold 1228.8
Red line y: 288
Blue line y: 192
[[[  0 288]]

 [[640 288]]]
ID: 1 crossed going down at Sat Apr 28 15:33:54 2018
ID: 2 crossed going down at Sat Apr 28 15:33:57 2018
ID: 3 crossed going down at Sat Apr 28 15:34:00 2018
ID: 5 crossed going up at Sat Apr 28 15:34:00 2018
ID: 7 crossed going down at Sat Apr 28 15:34:01 2018
ID: 11 crossed going down at Sat Apr 28 15:34:02 2018
ID: 9 crossed going up at Sat Apr 28 15:34:02 2018
ID: 7 crossed going up at Sat Apr 28 15:34:02 2018
ID: 13 crossed going down at Sat Apr 28 15:34:05 2018
EOF
UP: 3
DOWN: 6


In [6]:
p = {'1':{'gf':7},'2':{'x':5},'3':{'y':1}}

In [7]:
print(p)

{'1': {'gf': 7}, '2': {'x': 5}, '3': {'y': 1}}


In [11]:
keys = p.keys()

In [12]:
keys

dict_keys(['1', '2', '3'])

In [13]:
list(keys)

['1', '2', '3']

In [2]:
tracker = re3_tracker.Re3Tracker()

Restoring:
re3/conv1/W_conv -> [11, 11, 3, 96] = 0.139392MB
re3/conv1/b_conv -> [96] = 0.000384MB
re3/conv1_skip/W_conv -> [1, 1, 96, 16] = 0.006144MB
re3/conv1_skip/b_conv -> [16] = 6.4e-05MB
re3/conv1_skip/prelu -> [16] = 6.4e-05MB
re3/conv2/W_conv -> [5, 5, 48, 256] = 1.2288MB
re3/conv2/b_conv -> [256] = 0.001024MB
re3/conv2_skip/W_conv -> [1, 1, 256, 32] = 0.032768MB
re3/conv2_skip/b_conv -> [32] = 0.000128MB
re3/conv2_skip/prelu -> [32] = 0.000128MB
re3/conv3/W_conv -> [3, 3, 256, 384] = 3.538944MB
re3/conv3/b_conv -> [384] = 0.001536MB
re3/conv4/W_conv -> [3, 3, 192, 384] = 2.654208MB
re3/conv4/b_conv -> [384] = 0.001536MB
re3/conv5/W_conv -> [3, 3, 192, 256] = 1.769472MB
re3/conv5/b_conv -> [256] = 0.001024MB
re3/conv5_skip/W_conv -> [1, 1, 256, 64] = 0.065536MB
re3/conv5_skip/b_conv -> [64] = 0.000256MB
re3/conv5_skip/prelu -> [64] = 0.000256MB
re3/fc6/W_fc -> [74208, 2048] = 607.911936MB
re3/fc6/b_fc -> [2048] = 0.008192MB
re3/fc_output/W_fc -> [1024, 4] = 0.016384MB
re3/fc_ou

In [7]:
dir(Person)

['List',
 'Person',
 'Tuple',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'np']

In [12]:
p = Person.Person(1,1,1,1,1)

In [14]:
p.color_id_and_cout_of_person

2

In [4]:
p

{'1': <Person.Person.Person at 0x22399b769e8>,
 '2': <Person.Person.Person at 0x22399b76a58>,
 '3': <Person.Person.Person at 0x22399b76eb8>,
 '4': <Person.Person.Person at 0x22399b76cf8>,
 '5': <Person.Person.Person at 0x22399b76d30>,
 '6': <Person.Person.Person at 0x22399b76e48>,
 '7': <Person.Person.Person at 0x22399b76080>,
 '8': <Person.Person.Person at 0x22399b76828>}

In [8]:
p['1'].get_trace()

[(43, 272),
 (45, 291),
 (38, 332),
 (38, 337),
 (39, 324),
 (38, 327),
 (37, 327),
 (36, 330),
 (35, 332),
 (34, 333),
 (34, 339),
 (34, 342),
 (33, 348),
 (33, 355),
 (32, 362),
 (30, 367),
 (28, 371),
 (28, 375),
 (28, 377),
 (29, 382),
 (28, 383),
 (30, 383),
 (32, 384),
 (32, 384),
 (32, 381),
 (32, 381),
 (31, 385),
 (31, 386),
 (32, 388),
 (30, 390),
 (30, 393),
 (66, 427),
 (68, 426),
 (71, 422),
 (73, 418),
 (74, 416),
 (76, 415),
 (76, 415),
 (77, 415),
 (78, 414),
 (79, 411),
 (80, 407),
 (80, 402),
 (81, 395),
 (81, 389),
 (82, 383),
 (82, 376),
 (83, 371),
 (84, 368),
 (84, 366),
 (85, 364),
 (85, 362),
 (86, 360),
 (86, 358),
 (87, 356),
 (88, 354),
 (89, 351),
 (88, 349),
 (89, 346),
 (91, 340),
 (92, 335),
 (94, 331),
 (96, 324),
 (96, 318),
 (100, 311),
 (99, 307),
 (101, 304),
 (103, 299),
 (106, 300),
 (108, 298),
 (110, 296),
 (111, 293),
 (112, 291),
 (113, 289),
 (113, 288),
 (114, 285),
 (114, 281),
 (115, 277),
 (115, 270),
 (116, 266),
 (116, 260),
 (117, 255),

In [4]:
lines

{'blue_down_line': [(230, 262), (544, 205)],
 'intresting_zone': [(187, 29), (602, 29), (602, 360), (187, 360)],
 'red_up_line': [(220, 80), (575, 158)]}